In [4]:
import requests
import pandas as pd
from datetime import datetime
import os

# Your credentials (already set up for teamkilla)
APP_ID = '39d564fed2cf66b545b26173a75e9b81'
ACCOUNT_ID = '1000361546'
REGION = 'com'
FILE_NAME = 'wot_stats_history.csv'

# 1. Fetch the data from Wargaming
url = f"https://api.worldoftanks.{REGION}/wot/account/info/?application_id={APP_ID}&account_id={ACCOUNT_ID}"
response = requests.get(url).json()

if response['status'] == 'ok':
    player_data = response['data'][ACCOUNT_ID]
    all_stats = player_data['statistics']['all']
    
    # 2. Extract the stats you want
    record = {
        'Date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'Battles': all_stats['battles'],
        'Wins': all_stats['wins'],
        'Win_Rate': round((all_stats['wins'] / all_stats['battles']) * 100, 2),
        'Avg_Damage': round(all_stats['damage_dealt'] / all_stats['battles'], 0),
        'Global_Rating': player_data['global_rating']
    }
    
    # 3. Create a table (DataFrame) and show it
    df = pd.DataFrame([record])
    
    # 4. Save/Append to your CSV file for Tableau
    if not os.path.isfile(FILE_NAME):
        df.to_csv(FILE_NAME, index=False)
    else:
        df.to_csv(FILE_NAME, mode='a', header=False, index=False)
        
    print("Success! Here is your current data:")
    display(df)
else:
    print("Error fetching data. Check your API key or internet connection.")

Success! Here is your current data:


,Date,Battles,Wins,Win_Rate,Avg_Damage,Global_Rating
0,2026-02-23 23:41:01,12287,5285,43.01,492.0,3144


In [6]:
import requests
import pandas as pd
from datetime import datetime

# SETTINGS
APP_ID = '39d564fed2cf66b545b26173a75e9b81'
CLAN_ID = '1000067208'
REGION = 'com'

# 1. Get the list of all members in VALCO
# FIX: We now use the updated 'wot' API link instead of the deprecated 'wgn' one
clan_url = f"https://api.worldoftanks.{REGION}/wot/clans/info/?application_id={APP_ID}&clan_id={CLAN_ID}"

# We fetch the raw response first
response = requests.get(clan_url).json()

# SAFETY CHECK: Only proceed if Wargaming says "ok"
if response['status'] == 'ok':
    clan_data = response['data'][CLAN_ID]
    member_ids = [str(m['account_id']) for m in clan_data['members']]
    
    # 2. Get the stats for all members at once
    ids_string = ",".join(member_ids)
    stats_url = f"https://api.worldoftanks.{REGION}/wot/account/info/?application_id={APP_ID}&account_id={ids_string}"
    stats_data = requests.get(stats_url).json()['data']
    
    # 3. Organize the data for your Tableau Dashboard
    clan_stats = []
    for m_id, data in stats_data.items():
        if data:
            s = data['statistics']['all']
            clan_stats.append({
                'Player': data['nickname'],
                'Battles': s['battles'],
                'Win_Rate': round((s['wins'] / s['battles']) * 100, 2) if s['battles'] > 0 else 0,
                'Avg_Damage': round(s['damage_dealt'] / s['battles'], 0) if s['battles'] > 0 else 0,
                'Global_Rating': data['global_rating'],
                'Last_Battle': datetime.fromtimestamp(data['last_battle_time']).strftime('%Y-%m-%d')
            })

    # 4. Display the Clan Leaderboard
    df_valco = pd.DataFrame(clan_stats)
    df_valco.to_csv('valco_clan_stats.csv', index=False)
    print("VALCO Roster Updated successfully!")
    
    # Shows the top 10 players
    display(df_valco.sort_values(by='Global_Rating', ascending=False).head(10))
    
else:
    # If Wargaming sends an error, this prints exactly what went wrong
    print(f"API Error: {response.get('error')}")

VALCO Roster Updated successfully!


,Player,Battles,Win_Rate,Avg_Damage,Global_Rating,Last_Battle
44,paragate,12892,56.39,1662.0,8817,2026-01-20
92,Sloppapotamus,20365,54.78,1619.0,8748,2026-01-15
2,OY12,7285,54.34,1559.0,7884,2026-02-23
82,Leamund,50649,51.77,1424.0,7645,2026-02-23
20,Failedninja24,8329,52.31,1327.0,7207,2025-05-21
30,ClutchedCALIBER,17048,50.94,1579.0,7129,2026-02-21
39,Eluchill,19923,53.66,1208.0,7037,2026-01-22
91,MiniNuke_45,17229,53.47,1257.0,6915,2026-01-04
23,Lock_Load,48264,49.40,1603.0,6911,2026-02-21
62,pablovegachambe,30283,50.01,1311.0,6901,2026-02-21


In [9]:
%pip install gspread oauth2client


   ----- ---------------------------------- 1/8 [oauthlib]
   ----- ---------------------------------- 1/8 [oauthlib]
   ----- ---------------------------------- 1/8 [oauthlib]
   --------------- ------------------------ 3/8 [requests-oauthlib]
   -------------------- ------------------- 4/8 [oauth2client]
   ------------------------- -------------- 5/8 [google-auth]
   ------------------------- -------------- 5/8 [google-auth]
   ------------------------- -------------- 5/8 [google-auth]
   ----------------------------------- ---- 7/8 [gspread]
   ---------------------------------------- 8/8 [gspread]

Note: you may need to restart the kernel to use updated packages.


In [10]:
import requests
import pandas as pd
from datetime import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# --- 1. SETTINGS ---
APP_ID = '39d564fed2cf66b545b26173a75e9b81'
CLAN_ID = '1000067208'
REGION = 'com'

# --- 2. GET WARGAMING DATA ---
print("Fetching data from Wargaming...")
clan_url = f"https://api.worldoftanks.{REGION}/wot/clans/info/?application_id={APP_ID}&clan_id={CLAN_ID}"
response = requests.get(clan_url).json()

if response['status'] == 'ok':
    clan_data = response['data'][CLAN_ID]
    member_ids = [str(m['account_id']) for m in clan_data['members']]
    
    ids_string = ",".join(member_ids)
    stats_url = f"https://api.worldoftanks.{REGION}/wot/account/info/?application_id={APP_ID}&account_id={ids_string}"
    stats_data = requests.get(stats_url).json()['data']
    
    clan_stats = []
    for m_id, data in stats_data.items():
        if data:
            s = data['statistics']['all']
            clan_stats.append({
                'Player': data['nickname'],
                'Battles': s['battles'],
                'Win_Rate': round((s['wins'] / s['battles']) * 100, 2) if s['battles'] > 0 else 0,
                'Avg_Damage': round(s['damage_dealt'] / s['battles'], 0) if s['battles'] > 0 else 0,
                'Global_Rating': data['global_rating'],
                'Last_Battle': datetime.fromtimestamp(data['last_battle_time']).strftime('%Y-%m-%d')
            })

    df_valco = pd.DataFrame(clan_stats)
    print("Data processed successfully! Connecting to Google Cloud...")

    # --- 3. SEND DIRECTLY TO GOOGLE SHEETS ---
    # Give Python the keys to the bot
    scope = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name('valco_keys.json', scope)
    client = gspread.authorize(creds)
    
    # Open your specific spreadsheet
    sheet = client.open('VALCO_Clan_Stats').sheet1
    
    # Clear old data (this removes players who have left the clan)
    sheet.clear()
    
    # Upload the fresh data!
    sheet.update(range_name='A1', values=[df_valco.columns.values.tolist()] + df_valco.values.tolist())
    
    print("MISSION ACCOMPLISHED! Your Google Sheet is now populated.")

else:
    print(f"API Error: {response.get('error')}")

Fetching data from Wargaming...
Data processed successfully! Connecting to Google Cloud...
MISSION ACCOMPLISHED! Your Google Sheet is now populated.
